In [18]:
#Importing required libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
eps = np.finfo(float).eps
from numpy import log2 as log

In [19]:
train = pd.read_csv(r"C:\Users\Ruchi\Documents\titanic_train.csv")
test = pd.read_csv(r"C:\Users\Ruchi\Documents\titanic_test.csv")

In [20]:
org_train = train.copy()
org_test = test.copy()

In [21]:
data = train.append(test, sort = True)

In [22]:
#data.info()
total_pass = 1309
#print(data)

In [23]:
#data.head()

In [24]:
''' # Clean NA data
data.age.fillna(value = data.age.mean(), inplace = True)
data.fare.fillna(value = data.fare.mean(), inplace = True)
# Fill NA embarked values with the mode of embarked values
data.embarked.fillna(value=(data.embarked.value_counts().idxmax()), inplace=True)
data.survived.fillna(value=-1, inplace=True) 
data.boat.fillna(value = 0, inplace = True)
'''
data.dropna()
data['sex'] =  data['sex'].map( {'female': 0, 'male': 1} ).astype(int)
#Grouping ages
data.loc[data['age'] <= 16, 'age'] = 0
data.loc[(data['age'] > 16) & (data['age'] < 50), 'age'] = 1
data.loc[data['age'] >= 50, 'age'] = 2 

entropy_node = 0  #Initialize Entropy
values = data.survived.unique()  #Unique objects - 'Yes', 'No'
for value in values:
    fraction = data.survived.value_counts()[value]/len(data.survived)  
    entropy_node += -fraction*np.log2(fraction)

In [25]:
values = data.survived.value_counts()
print(values)

0.0    537
1.0    313
Name: survived, dtype: int64


In [26]:
537+313

850

In [27]:
dead = 537/850
alive = 313/850

In [29]:
e_surv = -(dead*log(dead)) - (alive*log(alive))
print(e_surv)

0.9493074611802688


In [33]:
values = data.age.value_counts(1)
print(values)

1.0    0.766730
0.0    0.128107
2.0    0.105163
Name: age, dtype: float64


In [44]:
e_age = -(0.766730*log(0.766730)) -(0.128107*log( 0.128107)) -(0.105163*log(0.105163))
print(e_age)

1.0153077174540548


In [35]:
num_young = len(data.loc[data['age'] == 0])
num_adult = len(data.loc[data['age'] == 1])
num_old = len(data.loc[data['age'] == 2])
print('Total Young:', num_young)
print('Total Adult:', num_adult)
print('Total Old:', num_old)

Total Young: 134
Total Adult: 802
Total Old: 110


In [36]:
#Conditional entropy of Young(0) subgroup
#e_young = females/total young , males/total young
young_f = len(data.loc[(data['age'] == 0) & (data['sex'] == 0)])/num_young
young_m = len(data.loc[(data['age'] == 0) & (data['sex'] == 1)]) /num_young
e_young = -(young_f*log(young_f)) - (young_m*log(young_m))
print(e_young)

0.9985532860835362


In [37]:
#Conditional entropy of Adult (1) subgroup
adult_f = len(data.loc[(data['age'] == 1) & (data['sex'] == 0)]) / num_adult
adult_m = len(data.loc[(data['age'] == 1) & (data['sex'] == 1)]) / num_adult
e_adult = -(adult_f*log(adult_f)) -(adult_m*log(adult_m))
print(e_adult)

0.9398319100995356


In [38]:
#Conditional entropy of Old (2) subgroup
old_f = len(data.loc[(data['age'] == 2) & (data['sex'] == 0)]) / num_old
old_m = len(data.loc[(data['age'] == 2) & (data['sex'] == 1)]) / num_old
e_old = -(old_f*log(old_f)) -(old_m*log(old_m))
print(e_old)

0.9299429352091801


In [46]:
#Weighted avg entropy of age subgroups = total young / total passengers, total adult/ tot pass, total old/tot pass
ratio_y = num_young/total_pass
ratio_a = num_adult/total_pass
ratio_o = num_old/total_pass
e_age_all = -(ratio_y*log(ratio_y)) -(ratio_a*log(ratio_a))-(ratio_o*log(ratio_o))
print(e_age_all)

1.069883632917358


In [47]:
e_wt_age = (ratio_y*e_young) + (ratio_a*e_adult) + (ratio_o*e_old)
print(e_wt_age)

0.7561841521069758


In [48]:
print(e_age)

1.0153077174540548


In [49]:
info_gain_age = e_age - e_wt_age
print(info_gain_age)

0.25912356534707903
